# import dataset

In [2]:
import nltk
nltk.download('punkt')
import pandas as pd
from google.colab import drive
from nltk.tokenize import word_tokenize
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn import svm
from sklearn.linear_model import SGDClassifier

from joblib import dump, load
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import itertools
import numpy as np

import nltk
from nltk import tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
stop=set(stopwords.words('english'))
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [4]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [5]:
colnames=['Label', 'Text'] 
path = '/content/drive/MyDrive/CS4248/fulltrain.csv'
X_train = pd.read_csv(path,header = None, names = colnames)

In [ ]:
X_train.head()

,Label,Text
0,1,"A little less than a decade ago, hockey fans w..."
1,1,The writers of the HBO series The Sopranos too...
2,1,Despite claims from the TV news outlet to offe...
3,1,After receiving 'subpar' service and experienc...
4,1,After watching his beloved Seattle Mariners pr...


# split the data into train and test?

what is the purpose of the xdev file,why my dimension is different from others?

In [6]:
from sklearn.model_selection import train_test_split

X1 = X_train['Text']
y = X_train['Label']


In [7]:
def preprocessing(document):

        # no lower case, no stopword removal, no lemmatization
        text = document

        # lowercase
        # text = document.lower()
        
        # lowercase with lemmatization
        # document = document.lower()
        # text_split = tokenize.word_tokenize(document)
        # text = [wnl.lemmatize(word) for word in text_split]
        # text =  " ".join(text)

        # lowercase with stop removal
        # document = document.lower()
        # text_split = tokenize.word_tokenize(document)
        # text = [word for word in text_split if word not in stop]
        # text = " ".join(text)

        #lower case with stop removal and lemmatization
        # document = document.lower()
        # text_split = tokenize.word_tokenize(document)
        # text = [word for word in text_split if word not in stop]
        # text = [wnl.lemmatize(word) for word in text]
        # text = " ".join(text)
        return text

In [21]:
X = [preprocessing(x) for x in X1]

In [22]:
X= pd.DataFrame(X)

In [23]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.4, random_state=42)
for train_index, test_valid_index in split.split(X,y):
    X_train = X.iloc[train_index][0]
    y_train = y.iloc[train_index]
    X_test_val = X.iloc[test_valid_index][0]
    y_test_val = y.iloc[test_valid_index]

split2 = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=42)
for test_index, valid_index in split2.split(X_test_val, y_test_val):
    X_tes = X_test_val.iloc[test_index]
    y_tes = y_test_val.iloc[test_index]
    X_val = X_test_val.iloc[valid_index]
    y_val = y_test_val.iloc[valid_index]

    

In [20]:
X_train.head()

48077    Goldman Sachs, the global investment banking a...
1092     Local hair stylist Pam Nowicki would love to d...
38920    Tennis: Roger Federer defeats Rafael Nadal to ...
46596    Russia and the United States welcomed signals ...
36659    Another Banker Found Dead Under Strange Circum...
Name: 0, dtype: object

In [16]:
y_train.head()

48077    4
1092     1
38920    4
46596    4
36659    3
Name: Label, dtype: int64

# tf-idf

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer(lowercase = False, ngram_range=(1,2), analyzer='word', max_features= 8000)

In [25]:
X_train_tfidf = tfidf_vect.fit_transform(X_train)

In [26]:
X_val_tfidf = tfidf_vect.transform(X_val)

In [27]:
X_test_tfidf   = tfidf_vect.transform(X_tes)

# NB

In [28]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
classifier = MultinomialNB()

classifier.fit(X_train_tfidf, y_train)
NB_pre = classifier.predict(X_test_tfidf)
from sklearn import metrics
print(metrics.confusion_matrix(y_tes,NB_pre))

[[2589   19  109   92]
 [  50 1289   42    8]
 [  66   52 3426   30]
 [ 301   10   74 1614]]


In [29]:
print(metrics.classification_report(y_tes,NB_pre))

              precision    recall  f1-score   support

           1       0.86      0.92      0.89      2809
           2       0.94      0.93      0.93      1389
           3       0.94      0.96      0.95      3574
           4       0.93      0.81      0.86      1999

    accuracy                           0.91      9771
   macro avg       0.92      0.90      0.91      9771
weighted avg       0.91      0.91      0.91      9771



In [30]:
print(metrics.accuracy_score(y_tes,NB_pre))

0.9127008494524613


In [31]:
from sklearn import metrics
print(f"F1: {metrics.f1_score(y_tes,NB_pre, average='macro')}")
print(f"Accuracy: {metrics.accuracy_score(y_tes,NB_pre)}")

F1: 0.9089089431392896
Accuracy: 0.9127008494524613
